# Installing Dependencies

In [ ]:
!python3 -m pip install pymupdf openai chromadb tqdm nltk tiktoken python-docx langchain langsmith langchain_openai moviepy streamlit

from openai import OpenAI
from tqdm import tqdm
import os

## Setting up Langsmith for Evaluation

In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "Java TA Chatbot"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Extract Text Function

In [3]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file.
    Args:
        pdf_path (str): The file path to the PDF document from which text needs to be extracted.
    Returns:
        str: A string containing all the text extracted from the PDF document.
    """
    
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

## Chunking Text for Efficient Processing

In this section, we will define a function to chunk text into smaller segments. This is particularly useful for handling large texts, allowing us to process them more efficiently and effectively. The function will break the text into overlapping chunks based on a specified maximum token limit and overlap size. This approach ensures that we maintain context across chunks while adhering to token constraints.

In [ ]:
import nltk

# Fix SSL issue (macOS sometimes has missing certs)
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Download punkt
nltk.download('punkt')

In [ ]:
import nltk

# Explicitly add the punkt path
# nltk.data.path.append("/Users/mohsintanveer/nltk_data")

from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

# Confirm it works
text = "Hello world. This is a test sentence. Let's see if it works."
print(sent_tokenize(text))

In [ ]:
import nltk
import tiktoken

nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

# Load tokenizer for OpenAI's embedding model
encoding = tiktoken.encoding_for_model("text-embedding-3-small")

def chunk_text(text, max_tokens=500, overlap=50):
    """
    Chunks the input text into smaller segments based on a maximum token limit, allowing for overlapping sentences for context.
    Args:
        text (str): The input text to be chunked.
        max_tokens (int, optional): The maximum number of tokens allowed in each chunk. Default is 500.
        overlap (int, optional): The number of tokens to overlap between consecutive chunks for context. Default is 50.
    Returns:
        List[str]: A list of text chunks, each containing sentences that do not exceed the specified token limit.
    Notes:
        - The function uses sentence tokenization to break the input text into sentences.
        - It maintains a running count of tokens to ensure that each chunk does not exceed the specified limit.
        - Overlapping sentences are included in the new chunk to provide context for the next segment.
        - The function handles the last chunk separately to ensure no text is left unchunked.
    """

    sentences = sent_tokenize(text)  # Break the text into individual sentences
    chunks = []                     # To store final chunks
    current_chunk = []              # Temporarily hold sentences for a chunk
    current_tokens = 0              # Running token count for the current chunk

    for sentence in sentences:
        token_count = len(encoding.encode(sentence))  # Tokens in this sentence

        # If adding this sentence exceeds the token limit:
        if current_tokens + token_count > max_tokens:
            chunks.append(" ".join(current_chunk))  # Save current chunk

            # Handle overlap: keep the last few sentences for context
            overlap_sents = []
            overlap_tokens = 0
            for s in reversed(current_chunk):  # Go backwards through the chunk
                s_tokens = len(encoding.encode(s))
                if overlap_tokens + s_tokens <= overlap:
                    overlap_sents.insert(0, s)  # Add sentence at the beginning
                    overlap_tokens += s_tokens
                else:
                    break  # Stop when overlap token limit is reached

            # Start a new chunk with overlapping sentences + the new one
            current_chunk = overlap_sents + [sentence]
            current_tokens = sum(len(encoding.encode(s)) for s in current_chunk)
        else:
            # Add this sentence to the current chunk
            current_chunk.append(sentence)
            current_tokens += token_count

    # Don't forget to save the last chunk if there's anything left
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

## Parsing Questions from Exercises

In this section, we define a function `parse_questions_from_exercises` that extracts exercise questions from a given text. The function identifies questions that begin with specific patterns, such as "Q1.", "Q2.", "Q5a.", etc. It returns a list of cleaned question strings, making it easier to work with the extracted questions in subsequent analysis or processing.

In [1]:
import re

def parse_questions_from_exercises(text):
    """
    Parse questions from a given text that contains exercise lines.
    This function takes a string input, splits it into lines, and extracts questions 
    that are identified by specific patterns (e.g., Q1., Q2., Q5a., Q8.). It handles 
    continuation lines that are part of the same question.
    Args:
        text (str): A string containing the text from which questions will be parsed.
    Returns:
        list: A list of strings, each representing a parsed question.
    """
    
    lines = text.split('\n')
    questions = []
    current_question = ""

    # Regex to match question identifiers like Q1., Q2., Q5a., Q8.
    question_start = re.compile(r"^(Q\d+(\.\d+)?[a-z]?\.)\s+(.*)")

    for line in lines:
        line = line.strip()
        if not line:
            continue

        match = question_start.match(line)
        if match:
            if current_question:
                questions.append(current_question.strip())
            current_question = match.group(3)  # the question text
        else:
            current_question += " " + line  # append continuation lines

    if current_question:
        questions.append(current_question.strip())

    return questions

In [6]:
text = extract_text_from_pdf("Exercises1-2.pdf")
text += " \n\n" + extract_text_from_pdf("Exercises2.pdf")
text += " \n\n" + extract_text_from_pdf("Exercises3.pdf")
text += " \n\n" + extract_text_from_pdf("Exercises4.pdf")
text += " \n\n" + extract_text_from_pdf("Exercises5.pdf")
questions = parse_questions_from_exercises(text)

## Answer Generation for Exercise Questions

In this section, we define a function `generate_exercise_answer` that utilizes the OpenAI API to generate answers for exercise questions. The function takes a question as input and optionally accepts a context prompt to provide additional information for the answer generation.

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def generate_exercise_answer(question, context_prompt=None):
    """
    Generates an answer to a Java exercise question using a chat-based AI model.
    Args:
        question (str): The Java exercise question that needs to be answered.
        context_prompt (str, optional): Additional context or instructions to guide the AI's response. 
                                         Defaults to None.
    Returns:
        str: A clear and accurate answer to the provided Java exercise question.
    """
    
    system_msg = (
        "You are a helpful Java tutor that gives clear, accurate answers "
        "to university-level Java exercises."
    )

    if context_prompt:
        system_msg += " " + context_prompt

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": question}
        ]
    )
    return response.choices[0].message.content.strip()

In [ ]:
qa_chunks = []

for q in tqdm(questions, desc="Generating answers"):
    answer = generate_exercise_answer(q)
    qa_chunks.append(f"Q: {q}\nA: {answer}")

- Logging exercise questions and answers to word document for evaluation

In [ ]:
from docx import Document

def save_qa_chunks_to_docx(qa_chunks, filename="qa_output.docx"):
    """
    Save question and answer chunks to a DOCX file.
    This function takes a list of question and answer chunks, processes them, 
    and saves them into a DOCX file with a specified filename. Each question 
    and answer pair is formatted with a numbered list for questions and 
    standard paragraph formatting for answers. Malformed chunks are skipped 
    with a warning printed to the console.
    Args:
        qa_chunks (list): A list of strings where each string contains a 
                          question and answer formatted as "Q: question\nA: answer".
        filename (str): The name of the output DOCX file. Defaults to "qa_output.docx".
    Returns:
        None: The function saves the output directly to a file and does not return any value.
    Example:
        qa_chunks = [
            "Q: What is Python?\nA: Python is a programming language.",
            "Q: What is AI?\nA: AI stands for Artificial Intelligence."
        ]
        save_qa_chunks_to_docx(qa_chunks, "output.docx")
    """

    doc = Document()
    doc.add_heading("Exercise Questions and Answers", level=1)

    for chunk in qa_chunks:
        if chunk.startswith("Q:") and "\nA:" in chunk:
            q_part, a_part = chunk.split("\nA:", 1)
            question = q_part[2:].strip()
            answer = a_part.strip()

            doc.add_paragraph(f"Q: {question}", style="List Number")
            doc.add_paragraph(f"A: {answer}")
            doc.add_paragraph("\n\n")  # Add extra spacing between entries
        else:
            print("Skipping malformed chunk:", chunk)

    doc.save(filename)
    print(f"Saved {len(qa_chunks)} Q&A pairs to {filename}")

In [ ]:
save_qa_chunks_to_docx(qa_chunks)

In [ ]:
import chromadb

# Point to the same persistent storage directory
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Load the existing collection
collection = chroma_client.get_collection(name="knowledge-base6")

In [ ]:
from tqdm import tqdm

# 1. Embed new Q&A chunks with progress bar
embeddings = [
    client.embeddings.create(input=chunk, model="text-embedding-3-small").data[0].embedding
    for chunk in tqdm(qa_chunks, desc="Embedding Q&A chunks")
]

# 2. Add to existing collection with progress bar
for chunk, embedding in tqdm(zip(qa_chunks, embeddings), total=len(qa_chunks), desc="Storing in ChromaDB"):
    collection.add(
        documents=[chunk],
        embeddings=[embedding],
        ids=[str(hash(chunk))]  # Optional: use uuid5 if you want stability across sessions
    )

In [ ]:
import fitz  # PyMuPDF
import base64
import re
from openai import OpenAI
from tqdm import tqdm
from docx import Document
from pathlib import Path
from typing import List

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# A page must contain EITHER "[xx marks]" OR a line beginning with "n."
QUESTION_PAGE_RE = re.compile(
    r"\[\d+\s*marks?\]"          #  [10 marks]   or [2 mark]
    r"|"                             #  …or…
    r"^\s*\d+\.\s",              #  1. 2. 3. etc. at line start
    re.IGNORECASE | re.MULTILINE
)

# Instruction verbs / OOP keywords for a secondary sanity check
VALID_QUESTION_PATTERNS = [
    r"\b(write|explain|describe|compare|implement|calculate|state|list|give)\b",
    r"\b(java|uml|class|method|object|interface|recursion)\b",
]
VALID_Q_RE = re.compile("|".join(VALID_QUESTION_PATTERNS), re.IGNORECASE)


def extract_pages(pdf_path):
    """Extracts text + images from each page"""
    doc = fitz.open(pdf_path)
    page_data = []

    for i, page in enumerate(doc):
        text = page.get_text().strip()
        pix = page.get_pixmap(dpi=300)
        img_bytes = pix.tobytes("png")
        base64_img = base64.b64encode(img_bytes).decode("utf-8")
        page_data.append({"page": i + 1, "text": text, "image": base64_img})
    return page_data


def extract_questions_only(pdf_path: str | Path) -> List[str]:
    """
    Return a list of *question blocks* from the given PDF, skipping
    cover pages and general instructions.
    """
    pdf_path = Path(pdf_path)
    doc = fitz.open(pdf_path)
    all_pages = [page.get_text().strip() for page in doc]

    question_pages = [t for t in all_pages if QUESTION_PAGE_RE.search(t)]
    raw_blocks = re.split(r"\n(?=\d+\.)", "\n".join(question_pages))

    questions = [
        block.strip()
        for block in raw_blocks
        if block.strip() and VALID_Q_RE.search(block)
    ]
    return questions


def find_pages_for_question(question_text, pages):
    """Estimate which page(s) the question came from based on matching text"""
    matched = []
    for page in pages:
        if question_text[:40] in page["text"]:
            matched.append(page)
    return matched


def answer_exam_questions(pdf_path, output_docx="java_exam_answers.docx"):
    pages = extract_pages(pdf_path)
    question_pages = [p for p in pages if QUESTION_PAGE_RE.search(p["text"])]
    question_blocks = extract_questions_only(pdf_path)

    # word_doc = Document()
    # word_doc.add_heading("Answers to Java Exam Questions", level=1)
    answers = []

    for idx, question_text in tqdm(
            enumerate(question_blocks, 1),
            total=len(question_blocks),
            desc="Answering questions"):

        matched_pages = find_pages_for_question(question_text, question_pages)
        image_base64 = matched_pages[0]["image"] if matched_pages else None

        messages = [
            {"role": "system", "content": (
                "You are a helpful and patient Java tutor at UCL. "
                "You are answering an exam question from a past paper. "
                "Explain all parts clearly, using beginner-friendly reasoning, and refer to UML diagrams if visible in the image."
                "For coding answers: 1) Add thorough but not redundant commenting explaining important code logic. 2) Try to use as beginner-friendly code keeping in mind this is for university level programming students."
            )},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question_text},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}} if image_base64 else {"type": "text", "text": "(No diagram found for this question.)"}
                ]
            }
        ]

        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0.3
            )
            answer = response.choices[0].message.content.strip()
        except Exception as e:
            answer = f"⚠️ Error generating answer: {str(e)}"

        answers.append((f"Question {idx}", question_text, answer))

        print(f"\n\n{'='*40}\n📝 Question {idx}\n{'='*40}")
        print(f"\n📄 {question_text[:300]}...\n\n✅ {answer[:500]}...\n")

    #     word_doc.add_heading(f"Question {idx}", level=2)
    #     word_doc.add_paragraph("📝 Question Text:", style="Intense Quote")
    #     word_doc.add_paragraph(question_text)
    #     word_doc.add_paragraph("✅ Answer:", style="Intense Quote")
    #     word_doc.add_paragraph(answer)
    #     word_doc.add_paragraph("\n")

    # word_doc.save(output_docx)
    # print(f"\n✅ All answers saved to: {output_docx}")
    return answers

In [ ]:
import os
from openai import OpenAI
import chromadb

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
# chroma_client = chromadb.PersistentClient(path="./chroma_db")
# collection = chroma_client.get_or_create_collection("knowledge-base6")

# Get all PDFs in your directory
directory = "./knowledge_base_data/past_papers"
pdf_files = [f for f in os.listdir(directory) if f.endswith(".pdf")]

all_chunks = []
all_metadatas = []

# Loop through all PDF files
for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
    full_path = os.path.join(directory, pdf_file)
    qa_pairs = answer_exam_questions(full_path)

    for q_id, question, answer in qa_pairs:
        combined_text = f"{q_id}\n\nQUESTION:\n{question}\n\nANSWER:\n{answer}"
        chunks = chunk_text(combined_text)

        all_chunks.extend(chunks)
        all_metadatas.extend([{"source": pdf_file, "question_id": q_id}] * len(chunks))

# Embed all chunks
embedding_model = "text-embedding-3-small"
embeddings = [
    client.embeddings.create(input=chunk, model=embedding_model).data[0].embedding
    for chunk in tqdm(all_chunks, desc="Embedding Chunks")
]

# Store all in ChromaDB
for chunk, embedding, metadata in tqdm(zip(all_chunks, embeddings, all_metadatas), total=len(all_chunks), desc="Storing in DB"):
    collection.add(
        documents=[chunk],
        embeddings=[embedding],
        ids=[str(hash(chunk))],
        metadatas=[metadata]
    )

# Embed and Store in Vector DB

In [ ]:
import os
from openai import OpenAI
import chromadb

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
# Create/load local persistent DB folder
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection("knowledge-base6")

# Get all PDFs in your directory
pdf_files = [f for f in os.listdir() if f.endswith(".pdf")]

all_chunks = []
all_metadatas = []

# Loop through all PDF files
for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
    text = extract_text_from_pdf(pdf_file)
    chunks = chunk_text(text)

    all_chunks.extend(chunks)
    # Save which file each chunk came from
    all_metadatas.extend([{"source": pdf_file}] * len(chunks))

# Embed all chunks
embedding_model = "text-embedding-3-small"
embeddings = [
    client.embeddings.create(input=chunk, model=embedding_model).data[0].embedding
    for chunk in tqdm(all_chunks, desc="Embedding Chunks")
]

# Store all in ChromaDB
for chunk, embedding, metadata in tqdm(zip(all_chunks, embeddings, all_metadatas), total=len(all_chunks), desc="Storing in DB"):
    collection.add(
        documents=[chunk],
        embeddings=[embedding],
        ids=[str(hash(chunk))],
        metadatas=[metadata]
    )

Storing in DB: 100%|██████████| 490/490 [00:03<00:00, 132.97it/s]


## Generate an Answer with RAG

In [ ]:
from langsmith import traceable
@traceable(name="RAG_Chatbot_Answer")
def rag_answer(query, collection, embedding_model="text-embedding-3-small", k=3):

    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
    
    # Step 1: Embed the user's question
    query_embedding = client.embeddings.create(
        input=query,
        model=embedding_model
    ).data[0].embedding

    # Step 2: Retrieve top-k similar chunks from ChromaDB
    results = collection.query(query_embeddings=[query_embedding], n_results=k)
    relevant_chunks = results["documents"][0]

    # Step 3: Build the RAG prompt
    context = "\n\n".join(relevant_chunks)
    prompt = f"""
You are a helpful Java teaching assistant at UCL. Use the context below, which is taken from course materials, to answer the user's question. If the answer is not in the context, say you don’t know.

Context:
{context}

Question:
{query}

Answer:
"""

    # Step 4: Ask GPT to answer using the context
    response = client.chat.completions.create(
        model="gpt-4o",  # or "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "You are a helpful Java teaching assistant at UCL."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    
    return response.choices[0].message.content

## Main function which calls the LLM

In [24]:
from langchain_openai import ChatOpenAI
from langsmith import traceable

@traceable(name="RAG_Chatbot_Answer")
def rag_answer2(query, collection, embedding_model="text-embedding-3-small", k=3):
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # keep for embeddings

    # Step 1: Embed the user's question
    query_embedding = client.embeddings.create(
        input=query,
        model=embedding_model
    ).data[0].embedding

    # Step 2: Retrieve top-k chunks from Chroma
    results = collection.query(query_embeddings=[query_embedding], n_results=k)
    relevant_chunks = results["documents"][0]

    # Step 3: Construct prompt
    context = "\n\n".join(relevant_chunks)
    full_prompt = f"""
You are a helpful and expert Java teaching assistant at UCL. You assist students by answering their questions using only the course material provided in the context.
Your answers must always be:
Accurate, based solely on the context below;
Thorough, with clear explanations and examples when relevant;
Friendly and pedagogical, like a knowledgeable TA during office hours.
🔍 Context Usage Instructions:
If the user asks you to generate **new teaching materials** (exam papers, quizzes, exercises, sample projects), you should **synthesize** them using the topics, code examples, and explanations from the context—even if no exact exam exists there.
If the user explicitly asks you to draw or create a UML diagram, you may rely on the UML Diagrams (Usage Guidelines) section in this prompt—even though no UML lives in the context.
Otherwise, use only the information found in the context. Do not invent APIs, methods, definitions, or facts.
You may reformat, rename, and adapt examples from the context to answer the user’s question.
Only if you’ve **tried both** factual lookup *and* generative synthesis (where allowed), **then** say:
    “Sorry, I couldn’t find that in the course material I was given.” and follow up with some counter questions related to the user question to make the user help you understand their question better.
Do not include this apology if you’ve already answered the question or explained something from the context.
📋 Answer Format:
Brief Summary
A one- or two-line direct answer to the question.
Detailed Explanation
A clear and structured explanation using the terminology and style of the UCL course.
Java Code (if relevant)
Provide working and formatted code blocks in:
```java
// Code with meaningful comments
public int square(int x) {
    'return x * x;'
}
```
Add comments or labels like // Constructor or // Method call example where helpful.
Edge Cases & Pitfalls
Briefly mention any exceptions, compiler warnings, gotchas, or common mistakes related to the topic.
Optional Extras (only if helpful)
ASCII-style diagrams for control flow, object relationships, or memory
Small tables (e.g., lifecycle states, type conversions)

🧩 📐 UML Diagrams (Usage Guidelines)
When a question involves object-oriented design, class structure, inheritance, interfaces, or relationships between multiple classes, you may include a simple UML diagram to illustrate the structure.
✅ Use UML when:
A student asks about class relationships (e.g., "How do these classes relate?")
A concept involves inheritance, interfaces, composition, or abstract classes
You are explaining object-oriented design patterns (e.g., Strategy, Factory, etc.)
A student specifically asks you to create/draw a UML diagram
✅ Format:
Use ASCII-style UML diagrams that clearly show class names, inheritance, fields, and methods
Keep diagrams minimal and clean — no need to use full UML syntax or notation
✅ Examples:

Inheritance Relationship:
+----------------+
|    Animal      |
+----------------+
| - name: String |
+----------------+
| +speak(): void |
+----------------+
        ▲
        |
+----------------+
|     Dog        |
+----------------+
| +bark(): void  |
+----------------+

Interface Implementation:

+--------------------+
|   Flyable          |
+--------------------+
| +fly(): void       |
+--------------------+

        ▲ implements
        |
+----------------+
|     Bird       |
+----------------+
| - wings: int   |
| +fly(): void   |
+----------------+

Composition:

+-------------------+
|     House         |
+-------------------+
| - address: String |
+-------------------+
| +build(): void    |
+-------------------+
        ◆
        |
+-------------------+
|     Room          |
+-------------------+
| - size: int       |
+-------------------+

Big UML Diagram Example:

                         ┌──────────────────────────┐
                         │        Employee          │
                         ├──────────────────────────┤
                         │ - name        : String   │
                         │ - department  : String   │
                         │ - monthlyPay  : int      │
                         ├──────────────────────────┤
                         │ +String getName()        │
                         │ +String getDepartment()  │
                         │ +int    getMonthlyPay()  │
                         └──────────────────────────┘
                                   ▲
                ┌──────────────────┴──────────────────┐
                │                                     │
     ┌──────────────────────────┐        ┌──────────────────────────┐
     │         Manager          │        │          Worker          │
     ├──────────────────────────┤        ├──────────────────────────┤
     │ - bonus        : int     │        │ (no extra fields)        │
     ├──────────────────────────┤        ├──────────────────────────┤
     │ +int getMonthlyPay()     │        │                          │
     └──────────────────────────┘        └──────────────────────────┘
                  ▲ 0..* (managed by ExecutiveTeam)
                  │
                  │
                  │               1
        ┌──────────────────────────┴──────────────────────────┐
        │                   ExecutiveTeam                     │
        ├──────────────────────────────────────────────────────┤
        │ +void add(Manager manager)                          │
        │ +void remove(String name)                           │
        └──────────────────────────────────────────────────────┘
                  ▲ 1 (created/owned by Company)
                  │
                  │
                  │
   ┌───────────────────────────────────────────────────────────────┐
   │                           Company                            │
   ├───────────────────────────────────────────────────────────────┤
   │ - name : String                                               │
   ├───────────────────────────────────────────────────────────────┤
   │ +void addWorker(String name, String department, int pay)      │
   │ +void addManager(String name, String department, int pay,     │
   │                      int bonus)                               │
   │ +void addToExecutiveTeam(Manager manager)                     │
   │ +int  getTotalPayPerMonth()                                   │
   └───────────────────────────────────────────────────────────────┘
                   | 1
                   | has
                   | 0..*
                   ▼
        ┌──────────────────────────┐
        │        Employee          │  (same box as above; association shown here)
        └──────────────────────────┘

✅ Explain the diagram in words:
“In this example, Dog inherits from Animal. The base class provides the speak() method, and Dog adds a new method bark().”
❌ Don’t use UML for simple method questions or unrelated procedural logic.

Mini Quiz (optional)
Occasionally include a short quiz question to reinforce learning (e.g., “What would happen if the return type was void?”). Include answers at the end.
✏️ Formatting Rules:
Use correct Java identifier formatting (e.g., MyClass, toString(), ArrayList<Integer>)
Use bullet points or subheadings where clarity improves
Do not include material or Java APIs not explicitly referenced in the context
⚠️ Handling Common Cases:
If the user question is too vague, explain a general case using course-relevant examples (e.g., square(int x) or sayHello()).
If multiple interpretations of a question are possible, briefly list the plausible ones and address each.
If the question mentions a Java keyword (e.g., final, static, record), define it precisely and relate it to context.
If the question is about bugs, compilation errors, or design, point to patterns, methods, or design tips from the context material.
🎓 Teaching Style:
Be professional, supportive, and clear — like a trusted lab demonstrator or tutor.
Prioritize conceptual clarity over fancy language.
Avoid filler. Never speculate.
Structure your answer to help students understand, not just memorize.
🧠 Self-Check Before Answering:
Ask yourself: 1. "If it is a UML diagram, use examples in your prompt and answer."
              2. “Else, can I find any relevant example, definition, or code in the context or the prompt that helps answer this question?”
If yes, adapt and use it.
If no, say: “Sorry, I couldn’t find that in the course material I was given.” and follow up with some counter questions related to the user question to make the user help you understand their question better.

Context:
{context}

Question:
{query}

Answer:
"""

    # Step 4: LangSmith-traceable LLM call
    llm = ChatOpenAI(model="gpt-4o", temperature=0.3)
    response = llm.invoke(full_prompt)

    return response.content

In [8]:
print(rag_answer2("Explain loops in Java", collection))

Brief Summary
Java provides three main types of loops: `while`, `do-while`, and `for` loops, each serving different purposes for controlling the flow of execution based on conditions.

Detailed Explanation
1. **While Loop**: 
   - The `while` loop repeatedly executes a block of code as long as the specified boolean expression evaluates to `true`. The loop body can be executed zero or more times.
   - Syntax:
     ```java
     while (boolean-expression) {
         // Loop body
     }
     ```
   - Example:
     ```java
     int counter = 0;
     while (counter < 10) {
         System.out.println("Hello " + counter);
         counter++;
     }
     ```
   - This loop prints "Hello" followed by numbers 0 through 9.

2. **Do-While Loop**:
   - The `do-while` loop is similar to the `while` loop but guarantees that the loop body is executed at least once because the boolean expression is evaluated after the loop body.
   - Syntax:
     ```java
     do {
         // Loop body
     } while (bo

In [23]:
# RAG test loop: Ask questions and get grounded answers
while True:
    query = input("\nAsk a question (or type 'exit' to quit): ")
    if query.lower() in {"exit", "quit"}:
        break
    answer = rag_answer2(query, collection)
    print("\nAnswer:\n", answer)


Answer:
 Sorry, I couldn’t find that in the course material I was given. Could you please provide more details or clarify your request? For example, are you looking for a specific type of UML diagram (e.g., class, sequence, component) or a particular scenario or system to model?


## Check if Vector Database is Working as Expected

In [6]:
import chromadb

chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_collection(name="knowledge-base6")
print("Total documents:", collection.count())
results = collection.get()

print(results)

Total documents: 631
{'ids': ['-5737636593685844336', '4161726037176896772', '-7015067473337148221', '5427849477021835599', '-2162375440861023478', '-82309721898439458', '-3808223688783965693', '779998593911956609', '-4966941535825223921', '213218592719752541', '-3667053665765989212', '-485897498936749062', '-7929762108824737640', '7136388715927165459', '2240196879119628726', '-7147468165870873829', '-8261321575821116554', '-7035468117437734956', '-400360422417883890', '6874013627418674059', '2297905522509143652', '-2335713254602839027', '-6453447699939945111', '-1974404212264744876', '-2286948535942619604', '7263514637533799110', '6938364598964664209', '-8322522218314387568', '4488844286822982597', '-7349586982697434852', '2626331865086036598', '-6757858797938694315', '7258875737566387841', '-3083731116894000265', '5366260443427792785', '4501614231566457780', '3778913231139023917', '-9154141468564381694', '4943687457784408780', '7927295193759219100', '1815728459430548446', '-137916413